In [ ]:
!pip install tensorflow
!pip install tensorflow_datasets
!pip install keras-self-attention

In [ ]:
#import tensorflow_datasets as tfds
import pandas as pd

#You can download the Dataset "ag_news_subset"
""""
# Download and prepare the AG News dataset
dataset, info = tfds.load('ag_news_subset', with_info=True)
# Print information about the dataset
print(info)
"""

#Or import it if you are already download it
# train_set = pd.read_csv('ag_news_subset.csv')
# test_set = pd.read_csv('ag_news_subset_test.csv')  #Validation set

train_set = pd.read_csv('ag_news_subset.csv', encoding='utf-8')
test_set = pd.read_csv('ag_news_subset_test.csv' , encoding='utf-8')

In [ ]:
train_set.head()

,description,label,title
0,b'AMD #39;s new dual-core Opteron chip is desi...,3,b'AMD Debuts Dual-Core Opteron Processor'
1,b'Reuters - Major League Baseball\\Monday anno...,1,"b""Wood's Suspension Upheld (Reuters)"""
2,b'President Bush #39;s quot;revenue-neutral q...,2,b'Bush reform may have blue states seeing red'
3,b'Britain will run out of leading scientists u...,3,"b""'Halt science decline in schools'"""
4,"b'London, England (Sports Network) - England m...",1,b'Gerrard leaves practice'


In [ ]:
test_set.head()

,description,label,title
0,"b' CHARLOTTE, N.C. (Sports Network) - Carolina...",1,"b""Carolina's Davis Done for the Season"""
1,"b' PRESENTACION, Philippines (Reuters) - Phili...",0,"b'Philippine Rebels Free Troops, Talks in Doubt'"
2,"b'SAN FRANCISCO, CA - November 30, 2004 -Ubiso...",3,b'New Rainbow Six Franchise for Spring 2005'
3,b'DANIEL VETTORI spun New Zealand to the brink...,1,b'Kiwis heading for big win'
4,"b""AFP - Georgian and South Ossetian forces ove...",0,"b'Shelling, shooting resumes in breakaway Geor..."


In [ ]:
"""
Label 0: “World”
Label 1: “Sports”
Label 2: “Business”
Label 3: “Sci/Tech”
"""

train_set['label'].unique()

array([3, 1, 2, 0])

In [ ]:
print(train_set.info())
print()
print()
print(test_set.info())

In [ ]:
X_train = train_set.drop('label' , axis = 1)
y_train = train_set['label']

X_test = test_set.drop('label' , axis = 1)
y_test = test_set['label']

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer , one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

In [ ]:
voc_size = 5000

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download("stopwords")

In [ ]:
#Data preprocessing
from nltk.stem.porter import PorterStemmer

def data_preprocessing(Data):
    ps = PorterStemmer()
    corpus = []

    for i in range (0 ,len(Data)):
        if (i % 1000 == 0):
            print(i)
        review = re.sub('[^a-zA-Z]' , ' ' , Data['title'][i])
        review = review.lower()
        review = review.split()

        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)

        corpus.append(review)

    return Embedding_data(corpus)

In [ ]:
from gensim.models import KeyedVectors

#use glove

def Embedding(corpus):
    # Load pre-trained GloVe vectors (e.g., 100-dimensional vectors)
    glove_model = KeyedVectors.load_word2vec_format('C:/glove/glove.6B.300d.txt', binary=False)

    # Convert words in the corpus to their GloVe embeddings
    embedded_docs = []
    for words in corpus:
        embeddings = [glove_model[word] for word in words if word in glove_model]
        embedded_docs.append(embeddings)

    return Padding(embedded_docs)


# #or
# def Embedding_data(corpus):
#     onehot_repr= [one_hot(words,voc_size) for words in corpus]
#     print(onehot_repr)

#     return Padding(onehot_repr)

In [ ]:
def Padding(onehot_repr):
    sent_length = 20
    embedded_docs = pad_sequences(onehot_repr , padding='post' , maxlen=sent_length)
    #print(embedded_docs)
    return embedded_docs

In [ ]:
X_train_final = np.array(data_preprocessing(X_train))
y_train_final = np.array(y_train)

X_test_final = np.array(data_preprocessing(X_test))
y_test_final = np.array(y_test)

len(X_train_final)

In [ ]:
from tensorflow.keras.layers import Embedding as KerasEmbedding
from keras_self_attention import SeqSelfAttention

embedding_vector_features=64
sent_length = 20
model = Sequential()
model.add(KerasEmbedding(input_dim = voc_size , output_dim = embedding_vector_features , input_length = sent_length))
model.add(Bidirectional(LSTM(100, return_sequences=True))) # Add return_sequences=True
model.add(SeqSelfAttention(attention_activation='sigmoid')) # Add attention layer
model.add(Dense(4 , activation = 'softmax'))

model.compile(loss = "categorical_crossentropy" , optimizer="adam" , metrics=['accuracy'], run_eagerly=True)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 64)            320000    
                                                                 
 bidirectional (Bidirection  (None, 200)               132000    
 al)                                                             
                                                                 
 dense (Dense)               (None, 4)                 804       
                                                                 
Total params: 452804 (1.73 MB)
Trainable params: 452804 (1.73 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# from tensorflow.keras import backend as K

# K.clear_session()

# # Now define your model and fit it


In [ ]:
from tensorflow.keras.utils import to_categorical

# Convert labels to one-hot encoding
y_train_final = to_categorical(y_train_final, num_classes=4)
y_test_final = to_categorical(y_test_final, num_classes=4)

In [ ]:
model.fit(X_train_final, y_train_final, validation_data=(X_test_final, y_test_final), epochs=10, batch_size=64)